## Assignment 1 

Elinor Funk, Leland Whitlock, Marlo Anzarut 

github: https://github.com/elinorfunk/compsoc_assignment1


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

C:\Users\whitl\AppData\Local\Temp\ipykernel_15748\1781237093.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
page = requests.get('https://ic2s2-2023.org/program')
bs = BeautifulSoup(page.content)
italics = bs.find_all('i')
name_list = []
for i in italics:
    names = i.get_text()
    clean_names = names.replace("Chair:", "")
    name_list += clean_names.split(', ')

In [4]:
for i in range(len(name_list)):
    name_list[i] = name_list[i].strip()
    name_list[i] = name_list[i].lower()

In [5]:
name_df = pd.DataFrame(name_list, columns=['names'])
no_dupes = name_df.drop_duplicates(keep='first', ignore_index=True)

In [7]:
no_dupes.to_csv("data/names.csv")

In [ ]:
# read author data from week 2, exercise 2
wd = os.getcwd()
data = wd[:wd.rfind("\\")] + "\\data"
data = data.replace("\\", "/")

part_one = pd.read_csv(f"{data}/part_two_info.csv")
part_two = pd.read_csv(f"{data}/part_two_info_2.csv")
complete_list = pd.concat([part_one, part_two], ignore_index=True)

final_df = complete_list.drop('Unnamed: 0', axis=1)

### Filtering/Works Requests


In [ ]:
# remove authors with works_count < 5, > 1000
non_contributors = final_df[(final_df.works_count < 5) | (final_df.works_count > 5000)].index
pmos = final_df.drop(non_contributors)
pmos = pmos.reset_index()

In [ ]:
# create the request url with the given authors and filters
def build_request(author_ids, cursor="*"):
    auth = "author.id:"
    for id in author_ids:
        auth += f"{id}|"
    auth = auth[:-1]


    concept_filter = "concept.id:https://openalex.org/C17744445|https://openalex.org/C144024400|https://openalex.org/C15744967|https://openalex.org/C162324750,concept.id:https://openalex.org/C121332964|https://openalex.org/C41008148|https://openalex.org/C33923547"
    url = f"https://api.openalex.org/works?per-page=200&cursor={cursor}&filter={auth},{concept_filter},cited_by_count:>10,authors_count:<10"
    return url

In [ ]:
# get a list of the authors used in the filter given a start index (max 25)
def get_authors(start):
    query_list = []
    if start + 24 > len(pmos):
        end = len(pmos) - 1
    else:
        end = start + 24
    for i in pmos.loc[start:end]["id"]:
        id = i[i.find('A'):]
        query_list.append(id)
    return query_list

In [ ]:
# parse the json of a work item in the html response
def get_work_row(item):
    id = item['id']
    pub_year = item['publication_year']
    cite_count = item['cited_by_count']
    authors = [item['authorships'][a]['author'].get('id') for a in range(len(item['authorships']))]
    title = item['title']
    aii = item['abstract_inverted_index']
    
    return [id, pub_year, cite_count, authors, title, aii]

In [ ]:
# refactor
works_df = pd.DataFrame(columns=['id', 'publication_year', 'cited_by_count', 'author_ids', 'title', 'abstract_inverted_index'])

In [ ]:
# fetch the next batch of results using the batch # (counter)
def fetch_results(counter):
    next_cursor = '*'

    while (next_cursor):
        # form request url
        r = build_request(get_authors(counter * 25), next_cursor)
        response = requests.get(r)
        if not response:
            break
        rjson = response.json()

        # set next_cursor for paging
        next_cursor = rjson['meta'].get('next_cursor')
        
        # add results of response to works_df
        for work in rjson['results']:
            works_df.loc[len(works_df.index)] = get_work_row(work)
    

In [ ]:
# get the works dataset by fetching all authors
for i in range(45):
    fetch_results(i)

works_df.drop_duplicates(subset='id')